Übung: Eventing mit K-native
-----------------------------

**Eventing** bietet eine flexible und erweiterbare Möglichkeit, Anwendungen zu entwickeln, die auf eine Vielzahl von Ereignissen reagieren können. 

**Eventing** entkoppelt Produzenten und Konsumenten von Ereignissen, was zu einer verbesserten Skalierbarkeit und Wartbarkeit führt.

**Knative Eventing** ist ein leistungsstarkes Framework innerhalb der Knative-Plattform, das es Entwicklern ermöglicht, ereignisgesteuerte Architekturen auf Kubernetes zu erstellen und zu verwalten.

#### Vorteile

* **Entkopplung**: Produzenten und Konsumenten von Ereignissen sind entkoppelt, was die Entwicklung und Wartung von Anwendungen vereinfacht.
* **Skalierbarkeit**: Automatische Skalierung der Anwendungen basierend auf der Anzahl der eingehenden Ereignisse.
* **Erweiterbarkeit**: Unterstützung für verschiedene Arten von Ereignisquellen und -senken, die leicht integriert werden können.
* **Cloud-agnostisch**: Funktioniert auf jeder Kubernetes-Umgebung, unabhängig vom Cloud-Anbieter.


- - -

Zuerst erstellen wir den Kubernetes Namespace



In [ ]:
import os
os.environ['NS_EVNT']='ms-evnt'
! kubectl create namespace ${NS_EVNT}
! # kubectl label  namespace ${NS_EVNT} istio-injection=enabled

In [ ]:
%%bash
kubectl apply --namespace ${NS_EVNT} -f https://gitlab.com/ch-mc-b/autoshop-ms/infra/kubernetes-templates/-/raw/main/3-2-0-deployment/catalog-deployment.yaml
kubectl apply --namespace ${NS_EVNT} -f https://gitlab.com/ch-mc-b/autoshop-ms/infra/kubernetes-templates/-/raw/main/3-2-0-deployment/customer-deployment.yaml
kubectl apply --namespace ${NS_EVNT} -f https://gitlab.com/ch-mc-b/autoshop-ms/infra/kubernetes-templates/-/raw/main/3-2-0-deployment/order-deployment.yaml
kubectl apply --namespace ${NS_EVNT} -f https://gitlab.com/ch-mc-b/autoshop-ms/infra/kubernetes-templates/-/raw/main/3-2-0-deployment/webshop-deployment.yaml 
kubectl apply --namespace ${NS_EVNT} -f https://gitlab.com/ch-mc-b/autoshop-ms/infra/kubernetes-templates/-/raw/main/3-2-0-deployment/catalog-service.yaml
kubectl apply --namespace ${NS_EVNT} -f https://gitlab.com/ch-mc-b/autoshop-ms/infra/kubernetes-templates/-/raw/main/3-2-0-deployment/customer-service.yaml
kubectl apply --namespace ${NS_EVNT} -f https://gitlab.com/ch-mc-b/autoshop-ms/infra/kubernetes-templates/-/raw/main/3-2-0-deployment/order-service.yaml
kubectl apply --namespace ${NS_EVNT} -f https://gitlab.com/ch-mc-b/autoshop-ms/infra/kubernetes-templates/-/raw/main/3-2-0-deployment/webshop-service.yaml
kubectl get   --namespace ${NS_EVNT} pods,services

In [ ]:
! echo "http://"$(cat ~/work/server-ip)":"$(kubectl get service --namespace ${NS_EVNT} webshop -o=jsonpath='{ .spec.ports[0].nodePort }')/webshop

### Ereignisquelle

Knative Eventing Sources sind Softwarekomponenten, die Ereignisse ausgeben. Die Aufgabe einer Quelle besteht darin, eine Verbindung zu Ereignissen herzustellen, diese abzuleiten, zu erfassen und ggf. zu puffern (häufig von einem externen System) und diese Ereignisse dann an den Sink weiterzuleiten .

Knative Eventing Sources installiert die folgenden Quellen:

In [ ]:
! kubectl api-resources --api-group='sources.knative.dev'

Sink(Empfänger)-Dienst bereitstellen

In [ ]:
%%bash
kn service create eventinghello \
  --concurrency-target=1 \
  --port=8080
  --image=quay.io/rhdevelopers/eventinghello:0.0.2

Ereignisquelle erstellen

In [ ]:
%%bash
kn source ping create eventinghello-ping-source \
  --schedule "*/2 * * * *" \
  --data '{"message": "Thanks for doing Knative Tutorial"}' \
  --sink ksvc:eventinghello

In [ ]:
! kn source ping list

In [ ]:
! stern eventinghello -c user-container -n knativetutorial

In [ ]:
! kubectl logs deployment/eventinghello-00001-deployment

- - -
Aufräumen

In [ ]:
%%bash
kn source ping delete eventinghello-ping-source
kn service delete eventinghello

**Sales** starten wie Serverless, weil dieser Microservice praktisch nie angesprochen wird.

In [ ]:
%%bash
kubectl patch configmap/config-domain --namespace knative-serving --type merge --patch '{"data":{"dukmaster-10-default.mshome.net":""}}' 
kn service create sales  --image registry.gitlab.com/ch-mc-b/autoshop-ms/app/management/sales:3.2.0  --port 8080 --namespace ${NS_EVNT}

Dann haben wir die klassischen Microservice wie **order**, **catalog** und **customer** welche als Standard Ressourcen (Pods, Services) in Kubernetes laufen und einen Serverless Microservice **sales**.

In [ ]:
! kubectl get --namespace ${NS_EVNT} pods,services
! kn service ls --namespace ${NS_EVNT} 

In [ ]:
%%bash
PORT=$(kubectl get service --namespace kourier-system kourier -o=jsonpath='{ .spec.ports[0].nodePort }')
curl -H "Host: sales.${NS_EVNT}.dukmaster-10-default.mshome.net" http://localhost:${PORT}/sales

Um die Services via Browser anzusprechen ist im Windows die Datei `C:\Windows\System32\Drivers\etc\hosts` um folgende Einträge zu ergänzen.

In [ ]:
%%bash
PORT=$(kubectl get service --namespace kourier-system kourier -o=jsonpath='{ .spec.ports[0].nodePort }')
echo $(cat ~/work/server-ip) shipment.${NS_EVNT}.dukmaster-10-default.mshome.net
echo $(cat ~/work/server-ip) invoicing.${NS_EVNT}.dukmaster-10-default.mshome.net
echo $(cat ~/work/server-ip) sales.${NS_EVNT}.dukmaster-10-default.mshome.net

echo ""
echo "Browser URLs"
echo http://shipment.${NS_EVNT}.dukmaster-10-default.mshome.net:${PORT}/shipment
echo http://invoicing.${NS_EVNT}.dukmaster-10-default.mshome.net:${PORT}/invoicing
echo http://sales.${NS_EVNT}.dukmaster-10-default.mshome.net:${PORT}/sales


Im Dashboard können wir das Verhalten beobachten. Nach einer gewissen Zeit werden die Pods **Shipment**, **Invoicing** und **Sales** beendet. Durch drücken von Refresh im Browser werden diese automatisch wieder gestartet.

In [ ]:
! echo "https://"$(cat ~/work/server-ip)":30443"

---
## Kiali — Observability


Kiali vom, Projekt Istio, hilft die Struktur (= Service Mesh) der Applikation und deren Microservices und Schnittstellen zu verstehen.


In der Oberfläche wechselt rechts auf "Graph" und wählt als Namespace **ms-faas** aus.

Mittels Aktivierung der Option "Traffic Animation" im Pulldown "Display" wird die Kommunkation sichtbar

In [ ]:
! kubectl get service -n istio-system -l app=kiali  -o yaml | sed 's/ClusterIP/NodePort/g' | kubectl apply -f -
! echo "Kiali   UI: http://"$(cat ~/work/server-ip)":"$(kubectl get -n istio-system service -l app=kiali -o=jsonpath='{ .items[0].spec.ports[0].nodePort }')

## Lasttest

Um die Verbindungen sichtbar zu machen, erzeugen wir ein wenig Traffic.

Dazu verwenden wir [hey](https://github.com/rakyll/hey) welche gleichzeitige parallele Request unterstützt.

In [ ]:
%%bash
PORT=$(kubectl get service --namespace kourier-system kourier -o=jsonpath='{ .spec.ports[0].nodePort }')
hey -n 10000 -c 10 -H "Host: sales.${NS_EVNT}.dukmaster-10-default.mshome.net" http://localhost:${PORT}/sales

- - -

Aufräumen


In [ ]:
! # kubectl delete pod --all --namespace ${NS-FAAS} --grace-period=0 --force
! kn service delete shipment  --namespace ${NS_EVNT}
! kn service delete invoicing --namespace ${NS_EVNT}
! kn service delete sales --namespace ${NS_EVNT}
! kubectl delete namespace ${NS_EVNT}

In [ ]:
! kn service delete sales --namespace ${NS_EVNT}